<a href="https://colab.research.google.com/github/SimoneBrancato/Magistrale-Informatica/blob/main/Simone/0103%20Big%20Data/Homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Esercizio (consegna il 7 aprile 2025)
Scaricare i dati disponibili a questo url: amazon reviews o questo amazon review dropbox
Il dataset è composto da due file: train and test in csv. Ogni file contiene le seguenti informazioni

polarity - 1 for negative and 2 for positive
title - review heading
text - review body

Generare i vettori sparsi applicando il q-shingle ai dati di training con q=3.
Sui vettori sparsi Applicare il MinHashing LSH sul dataset di training.

Usare il file di testing e applicare una k-nearest neighbor con i dati di testing su cui è stato applicato l'hashing. Usare k=3 e classificare l'elemento con del test set con la polarità maggiormente presente.

*Identificare i cluster di recensioni. Ogni cluster di recensione contiene le coppie di recensioni che hanno una similarità > di 0.6. Da svolgere dopo l'introduzione alle network

In [1]:
import kagglehub

path = kagglehub.dataset_download("kritanjalijain/amazon-reviews")

print("Path to dataset files:", path)

100%|██████████| 1.29G/1.29G [00:44<00:00, 31.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/kritanjalijain/amazon-reviews/versions/2


In [2]:
import pandas as pd
import os

print("File: ", os.listdir(path))

File:  ['amazon_review_polarity_csv.tgz', 'test.csv', 'train.csv']


In [3]:
q = 3

def shingle(text: str, k: int):
    shingle_set = []
    for i in range(len(text) - k+1):
        shingle_set.append(text[i:i+k])
    return set(shingle_set)

In [4]:
train_df = pd.read_csv("/root/.cache/kagglehub/datasets/kritanjalijain/amazon-reviews/versions/2/train.csv", header=None, names=["polarity", "title", "text"])
test_df = pd.read_csv("/root/.cache/kagglehub/datasets/kritanjalijain/amazon-reviews/versions/2/test.csv", header=None, names=["polarity", "title", "text"])

In [16]:
shingles_list= []

for _,row in train_df.iloc[:100].iterrows():
  full_text = f"{row['title']} {row['text']}"
  record_shingles = shingle(full_text, q)
  shingles_list.append(record_shingles)


In [25]:
# Mostriamo solo i primi 5, e massimo 10 shingle per vettore
for shingles in shingles_list[:5]:
    print("Shingles:", list(shingles)[:10])
    print("-----")

Shingles: ['end', 'e v', ' vi', 'ron', ' ba', 'tar', 'tun', 'e n', ' mi', ' tr']
-----
Shingles: ['ett', 's p', 's Y', 'I f', 'g o', 'ee ', '. I', 'din', 'o b', 'one']
-----
Shingles: ['ish', ' tw', 'ron', ', h', '), ', 'tar', 'o m', 't.I', 'e n', ' tr']
-----
Shingles: ['ish', 'end', 'e v', ' vi', 'ron', 's P', 'sho', 'wee', 'len', 'tar']
-----
Shingles: ['s p', 'ron', 'len', 'tar', 'rfe', 'hea', 's, ', 'var', 'rot', 'a d']
-----


In [27]:
vocab = set()

for shingles in shingles_list:
  vocab.update(shingles)
print(list(vocab)[:10])

['yss', 'MH ', 'et.', 'd v', 'ed]', 'dub', 'an"', ' ba', 'ulw', 'lpf']


In [28]:
# Creazione dei vettori sparsi per ogni chiave
sparse_vectors = []

for shingles in shingles_list:
  sparse_vectors.append([1 if x in shingles else 0 for x in vocab])

# Stampa un esempio
for vec in sparse_vectors[:5]:
    print(vec[:20])

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [29]:
# Adesso applichiamo il MinHash

from random import shuffle

hash_ex = list(range(1, len(vocab)+1))
shuffle(hash_ex)  # Randomizziamo il range
print(hash_ex[:10])

[3682, 1936, 2346, 5206, 5349, 1639, 5146, 834, 1884, 1631]
